In [58]:
import Data.Maybe
import Data.Foldable
import qualified Data.Map as Map

In [59]:
data Instruction = Instruction { target::String,
                                 op::Int -> Int,
                                 cond::String,
                                 predicate::Int -> Bool}

-- Parse lines in the form "c inc -20 if c == 10"
readLine :: String -> Instruction
readLine s =
    let ws = words s
        target_register = ws !! 0
        delta = read $ ws !! 2
        target_op = case (ws !! 1) of
             "inc" -> (+)
             "dec" -> (-)
        cond_register = ws !! 4
        cond_value = read $ ws !! 6
        cond_op = case (ws !! 5) of
             "==" -> (==)
             "!=" -> (/=)
             ">"  -> (>)
             ">="  -> (>=)
             "<"  -> (<)
             "<="  -> (<=)
        in Instruction target_register (flip target_op delta)
                       cond_register (flip cond_op cond_value)

In [60]:
type Machine = Map.Map String Int

run :: Machine -> Instruction -> Machine
run m i = Map.insert (target i)
    (if (predicate i) (get $ cond i)
     then (op i) (get $ target i)
     else get $ target i)
    m
  where get s = fromMaybe 0 $ Map.lookup s m

In [79]:
maximum . Map.elems . foldl run Map.empty . fmap readLine . lines <$> readFile "input8.txt"

8022

In [78]:
maximum . concatMap Map.elems . scanl run Map.empty . fmap readLine . lines <$> readFile "input8.txt"

9819